In [1]:
# Do not edit this cell

import os
java8_location= '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['JAVA_HOME'] = java8_location

!pip install pyspark==2.4.0
!pip install graphframes

os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages graphframes:graphframes:0.6.0-spark2.3-s_2.11 pyspark-shell")

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType
spark = SparkSession.builder.appName("Spark").master("local[4]").getOrCreate()

     |████████████████████████████████| 213.4MB 105kB/s 
     |████████████████████████████████| 204kB 39.3MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.0-py2.py3-none-any.whl size=213793601 sha256=21bb126540cd012fd71f2590b89137d7882ae101427c2cba7ec62e3f73065a0f
  Stored in directory: /root/.cache/pip/wheels/cd/54/c2/abfcc942eddeaa7101228ebd6127a30dbdf903c72db4235b23
Successfully built pyspark


In [0]:
# Begin your code from this cell. Use as many cells as you need.

# Custome schema for our Dataframe

schema = StructType([
    StructField("src", IntegerType(), True),
    StructField("dst", IntegerType(), True)])

# Read csv as DataFrame
edges = spark.read.csv("/content/net.txt",schema=schema)

In [3]:
print(type(edges))

edges.take(5)



<class 'pyspark.sql.dataframe.DataFrame'>


[Row(src=1, dst=2),
 Row(src=2, dst=3),
 Row(src=2, dst=4),
 Row(src=4, dst=1),
 Row(src=3, dst=5)]

In [4]:
# Getting verticed from edges by performing union of both the columns

verticesfromedges = edges.select('src').union(edges.select('dst')).withColumnRenamed('src','id')
print(verticesfromedges.take(5))
print(F"Count of Vertices - { verticesfromedges.count() }")

[Row(id=1), Row(id=2), Row(id=2), Row(id=4), Row(id=3)]
Count of Vertices - 12


In [5]:
# Getting distinct vertices

vertices = verticesfromedges.distinct()
print(F"Count of Unique Vertices - { vertices.count() }")

Count of Unique Vertices - 5


In [0]:
# Creating graph

from graphframes import GraphFrame
g = GraphFrame(vertices, edges)

In [7]:
g.vertices.show(5)

+---+
| id|
+---+
|  1|
|  3|
|  5|
|  4|
|  2|
+---+



In [8]:
g.edges.show(5)

+---+---+
|src|dst|
+---+---+
|  1|  2|
|  2|  3|
|  2|  4|
|  4|  1|
|  3|  5|
+---+---+
only showing top 5 rows



In [0]:
# Motif to find mutual freind

suggested_friend_by_mutual_friend_reletionship = g.find("(v1)-[e1]->(v2);(v2)-[e2]->(v3);!(v3)-[]->(v1);!(v1)-[]->(v3)")

In [10]:
suggested_friend_by_mutual_friend_reletionship.show(5)

+---+------+---+------+---+
| v1|    e1| v2|    e2| v3|
+---+------+---+------+---+
|[1]|[1, 2]|[2]|[2, 5]|[5]|
|[1]|[1, 2]|[2]|[2, 3]|[3]|
+---+------+---+------+---+



In [11]:
suggested_friend_by_mutual_friend_reletionship.select("v1.id","v3.id").withColumnRenamed("id","First_id").show(5)

+--------+--------+
|First_id|First_id|
+--------+--------+
|       1|       5|
|       1|       3|
+--------+--------+

